# Basic usage example

This is an example of how to use Paramaterial to process a dataset of uniaxial tensile test measurements.
The example follows a four-phase strategy, consisting of: data preparation, data processing, data aggregation and quality control.


## Data Preparation

The example dataset of tensile test measurements was published on Mendeley Data and is available at [DOI: 10.17632/rd6jm9tyb6.2](https://data.mendeley.com/datasets/rd6jm9tyb6/2). To get started with the example, download the data, unpack it, and save the CSV files in a directory `data/00 backup data`. Make a spreadsheet of metadata, save the spreadsheet at `info/00 backup info.xlsx`.

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read the Excel file into a pandas DataFrame
metadata = pd.read_excel('info/00 backup info.xlsx')
metadata.head(5)

,test_id,old filename,test type,material,temperature,rate
0,test_ID_001,P_020_F_1_017_198_18.csv,P,F,20,0.000866
1,test_ID_002,P_020_F_2_018_202_19.csv,P,F,20,0.000866
2,test_ID_003,P_020_F_3_019_208_25.csv,P,F,20,0.000866
3,test_ID_004,P_020_G_1_021_197_16.csv,P,G,20,0.000866
4,test_ID_005,P_020_G_2_022_195_24.csv,P,G,20,0.000866


Make sure to rename the datafiles according to the naming convention used in the `test_id` column of the metadata spreadsheet.


In [23]:
import shutil
import os

# Iterate through the rows of the metadata table
for index, row in metadata.iterrows():
    # Construct the old and new file names
    old_filename = "data/00 backup data/" + row['old filename']
    new_filename = "data/01 prepared data/" + row['test_id'] + ".csv"
    # Copy the file
    if not os.path.exists("data/01 prepared data/"):
        os.mkdir("data/01 prepared data/")
    shutil.copy(old_filename, new_filename)

# Copy the metadata to a new spreadsheet and drop the old filename column
pd.read_excel('info/00 backup info.xlsx').drop(columns=['old filename']).to_excel('info/01 prepared info.xlsx', index=False)

We are now done with setting up the data for the example, and can use the automated processing and analysis functionality provided by Paramaterial.

In [24]:
# Import Paramaterial
import paramaterial as pam

Load the data and metadata.

In [25]:
# Load the metadata spreadsheet and data files into a DataSet object
prepared_ds = pam.DataSet('info/01 prepared info.xlsx', 'data/01 prepared data')

# Get the metadata as a table
prepared_ds.info_table.head(5)

,test_id,test type,material,temperature,rate
0,test_ID_001,P,F,20,0.000866
1,test_ID_002,P,F,20,0.000866
2,test_ID_003,P,F,20,0.000866
3,test_ID_004,P,G,20,0.000866
4,test_ID_005,P,G,20,0.000866


For this example, we want to use only the tensile tests.

In [ ]:
# Filter to get only the tensile tests
prepared_ds = prepared_ds.subset({'test_type': ['T']})

# Sort the dataset
prepared_ds = prepared_ds.sort_by(['temperature', 'lot'])

### Make the experimental matrix
We want to identify useful groupings and make visualisations.

In [ ]:
# make a heatmap showing the distribution across lot and temperature
pam.experimental_matrix(prepared_ds, index='temperature', columns='lot', as_heatmap=True)

In [ ]:
import matplotlib as mpl

cmap_green = mpl.colors.LinearSegmentedColormap.from_list("", ["white", (0.2124, 0.3495, 0.1692)])

fig, ax = plt.subplots(1, 1, figsize=(4.2, 3.4))

pam.make_experimental_matrix(info_table=prepared_ds.info_table,
                             index='temperature', columns='lot',
                             as_heatmap=True, **dict(linewidths=2, cbar=False, annot=True, cmap=cmap_green, vmax=7,
                                                     annot_kws={'size': 10}, square=True))

ax.set_xlabel('Lot')
ax.set_ylabel('Temperature ($^{\circ}$C)')
ax.tick_params(rotation=0)

save('bu-exp-matrix')

### Visualise the prepared data.

We could colour by lot or by temperature. Colouring by lot is useful to observe variation, colouring by temperature is useful to observe temperature variation.

In [ ]:
styler = pam.plotting.Styler(
    color_by='temperature', color_by_label='(°C)', cmap='plasma', color_norm=plt.Normalize(20, 310)
).style_to(prepared_ds)

stress_strain_labels = dict(x='Strain', y='Stress_MPa', ylabel='Stress (MPa)')


def ds_plot(ds, title="", **kwargs):
    return pam.dataset_plot(ds=ds, styler=styler, **stress_strain_labels, title=title, **kwargs)


def ds_subplots(ds, **kwargs):
    axs = pam.dataset_subplots(
        ds=ds, shape=(6, 5), styler=styler, figsize=(10, 12), hspace=0.1, wspace=0.1,
        rows_by='temperature', row_vals=[[20], [100], [150], [200], [250], [300]],
        cols_by='lot', col_vals=[['A'], ['B'], ['C'], ['D'], ['E']],
        col_titles=[f'Lot {lot}' for lot in 'ABCDE'], subplots_adjust=0.02,
        **stress_strain_labels, **kwargs
    )
    for ax in axs.flat:
        ax.set_ylabel('Stress (MPa)')
    return axs

In [ ]:
ax = ds_plot(prepared_ds, xlim=(None, 0.35), title='Prepared Data')

ax.set_xticks(np.arange(0, 0.35, 0.1))
ax.set_xticklabels(['0', '10', '20', '30'])
ax.set_xlabel('Strain (%)')

save('bu-prepared')

In [ ]:
ax = ds_plot(prepared_ds, figsize=(9.2,7), xlim=(None, 0.31), title='Prepared Data')


ax.set_xticks(np.arange(0, 0.31, 0.05))
ax.set_xticklabels(['0', '5', '10', '15', '20', '25', '30'])
ax.set_xlabel('Strain (%)')

save('bu-single-plot')

In [ ]:
axs = ds_subplots(prepared_ds)

for ax in axs.flat:
    ax.set_xticks([0, 0.1, 0.2])
    ax.set_xticklabels(['0', '10', '20'])
    ax.set_xlabel('Strain (%)')

save('bu-prepared-subplots')

## Data Processing

### Find UTS and fracture point

In [ ]:
prepared_ds = pam.find_UTS(ds=prepared_ds, strain_key='Strain', stress_key='Stress_MPa')
prepared_ds = pam.find_fracture_point(ds=prepared_ds, strain_key='Strain', stress_key='Stress_MPa')

### Trimming

In [ ]:
def trim_to_small_strain(di):
    di.data = di.data[di.data['Strain'] < 0.01]
    return di


trimmed_ds = prepared_ds.apply(trim_to_small_strain)
ax = ds_plot(trimmed_ds, 'Trimmed Data')

ax.set_xticks([-0.002, 0, 0.002, 0.004, 0.006, 0.008, 0.01])
ax.set_xticklabels([-0.2, 0, 2, 4, 6, 8, 10])
ax.set_xlabel('Strain (%)')

save('bu-trimmed')

### Foot Correction

In [ ]:
corrected_ds = trimmed_ds.apply(pam.find_upl_and_lpl, preload=36, preload_key='Stress_MPa',
                                suppress_numpy_warnings=True).apply(pam.correct_foot)
ds_plot(corrected_ds, title='Corrected Data')

save('bu-foot-corrected')

### Foot correction screening

Make screening pdf.

In [ ]:
def foot_correction_screening_plot(di):
    temp = di.info['temperature']
    color = styler.color_dict[temp]
    UPL = (di.info['UPL_0'], di.info['UPL_1'])
    LPL = (di.info['LPL_0'], di.info['LPL_1'])
    _ax = ds_plot(trimmed_ds.subset({'test_id': di.test_id}), alpha=0.5)
    ds_plot(corrected_ds.subset({'test_id': di.test_id}), ax=_ax, figsize=(10, 5.8),
            title=f'{di.test_id}: {di.info["temperature"]}°C, Lot {di.info["lot"]}, {di.info["number"]}')
    _ax.axline(UPL, slope=di.info['E'], c=color, ls='--', alpha=0.5, zorder=500 + temp)
    _ax.plot(*UPL, c='k', mfc=color, marker=4, alpha=0.8, markersize=6, zorder=1000 + temp)
    _ax.plot(*LPL, c='k', mfc=color, marker=5, alpha=0.8, markersize=6, zorder=1000 + temp)

# foot_correction_screening_plot(corrected_ds[3])
# pam.make_screening_pdf(corrected_ds, foot_correction_screening_plot, 'info/foot correction screening.pdf');

In [ ]:
def foot_correction_screening_plot(di):
    color = styler.color_dict[di.info['temperature']]
    UPL = (di.info['UPL_0'], di.info['UPL_1'])
    LPL = (di.info['LPL_0'], di.info['LPL_1'])
    ax = ds_plot(trimmed_ds.subset({'test_id': di.test_id}), alpha=0.5)
    ds_plot(corrected_ds.subset({'test_id': di.test_id}), ax=ax,
            title=f'{di.test_id}: {di.info["temperature"]}°C, Lot {di.info["lot"]}, {di.info["number"]}')
    ax.axline(UPL, slope=di.info['E'], c=color, ls='--', alpha=0.5)
    ax.plot(*UPL, c=color, marker=4)
    ax.plot(*LPL, c=color, marker=5)


foot_correction_screening_plot(corrected_ds[3])
# pam.make_screening_pdf(corrected_ds, foot_correction_screening_plot, 'info/foot correction screening.pdf');

Reject flagged tests.

In [ ]:
rejected_ds = pam.read_screening_pdf_fields(corrected_ds, 'info/foot correction screening marked.pdf')
rejected_items = rejected_ds.info_table[rejected_ds.info_table['reject'] == '/Yes']
rejected_items[['test_id', 'temperature', 'lot', 'number', 'reject', 'comment']].to_latex(
    buf='info/02 rejected items.tex', index=False)

In [ ]:
screened_ds = pam.remove_rejected_items(corrected_ds, 'info/foot correction screening marked.pdf')
ds_plot(screened_ds, title='Screened Data')

save('bu-foot-corrected-screened')

### Calculate proof stress

In [ ]:
proof_ds = pam.find_proof_stress(ds=screened_ds, proof_strain=0.002, E_key='E',
                                 strain_key='Strain', stress_key='Stress_MPa')

### Write processed data

In [ ]:
proof_ds.write_output('info/02 processed info.xlsx', 'data/02 processed data')
processed_ds = pam.DataSet('info/02 processed info.xlsx', 'data/02 processed data')

In [ ]:
axs = ds_subplots(processed_ds)

for ax in axs.flat:
    ax.set_xticks([0, 0.005, 0.01])
    ax.set_xticklabels(['0', '0.5', '1'])
    ax.set_xlabel('Strain (%)')

save('bu-processed-subplots')

## Data Aggregation

### Representative curves

In [ ]:
# pam.make_representative_data(
#     processed_ds, 'info/03 temp repres info.xlsx', 'data/03 temp repres data',
#     repres_col='Stress_MPa', group_by_keys=['temperature', 'test_type'], interp_by='Strain',
#     group_info_cols=['UTS_0', 'UTS_1', 'FP_0', 'E', 'PS_0.002_0', 'PS_0.002_1', 'UPL_1'], interp_range='inner')

In [ ]:

temp_repres_ds = pam.DataSet('info/03 temp repres info.xlsx', 'data/03 temp repres data', test_id_key='repres_id')
ax = ds_plot(temp_repres_ds, fill_between=('down_std_Stress_MPa', 'up_std_Stress_MPa'), title='Representative Curves')
ax.set_xticks([0.002, 0.004, 0.006, 0.008, 0.010])
ax.set_xticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'])
ax.set_xlabel('Strain (%)')

save('bu-temp-repres')

In [ ]:
temp_repres_ds.info_table.to_latex(buf='info/repres_table.tex', index=False, formatters='?')

In [ ]:
# pam.make_representative_data(processed_ds, 'info/04 lot temp repres info.xlsx', 'data/04 lot temp data',
#                              repres_col='Stress_MPa', group_by_keys=['lot', 'temperature', 'test_type'],
#                              interp_by='Strain', interp_range='inner',
#                              group_info_cols=['UTS_0', 'UTS_1', 'FP_0', 'E', 'PS_0.002_0', 'PS_0.002_1'])

In [ ]:
lot_repres_ds = pam.DataSet('info/04 lot temp repres info.xlsx', 'data/04 lot temp data', test_id_key='repres_id')

In [ ]:
axs = ds_subplots(processed_ds)
for i, temp in enumerate(processed_ds.info_table['temperature'].unique()):
    for j in range(axs.shape[1]):
        ax = axs[i, j]
        ax.set_xticks([0, 0.005, 0.01])
        ax.set_xticklabels(['0', '0.5', '1'])
        ds_plot(ds=temp_repres_ds.subset({'temperature': temp}), ax=axs[i, j], alpha=0.3, plot_legend=False,
            fill_between=('up_std_Stress_MPa', 'down_std_Stress_MPa'))
        ax.set_xlabel('Strain (%)')

save('bu-temp-repres-subplots')

In [ ]:
ds_subplots(lot_repres_ds, fill_between=('down_std_Stress_MPa', 'up_std_Stress_MPa'));

save('bu-lot-repres-subplots')

## Fitting Models

In [ ]:
ramberg_ms = pam.ModelSet(
    model_func=pam.models.ramberg,
    var_names=['E', 'UPL_1'],
    param_names=['H', 'n'],
    bounds=[(0., 1000.), (0.01, 0.8)]
)

ramberg_ms.fit_to(temp_repres_ds, x_key='Strain', y_key='Stress_MPa')
ramberg_ds = ramberg_ms.predict()

In [ ]:
ramberg_ds.info_table

In [ ]:
ax = ds_plot(temp_repres_ds, alpha=0.5)
ds_plot(ramberg_ds, ls='--', ax=ax, title='Fitted Curves')
ax.set_xticks([0.002, 0.004, 0.006, 0.008, 0.010])
ax.set_xticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'])
ax.set_xlabel('Strain (%)')

save('bu-fitted-curves')

## Analysis and Comparison

## Quality Control